In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sys
import config

sys.path.append('model_building/create_image_folders.py')
from model_building.create_image_folders import * 
sys.path.append('model_building/keras_model.py')
from model_building.keras_model import *

import warnings
warnings.simplefilter("ignore", UserWarning)


## Load Data

In [ ]:
## Set paths
img_folder = "/home/jovyan/my_work/QB/image/images/"
train_img = "/home/jovyan/my_work/QB/image/train/"
val_img = "/home/jovyan/my_work/QB/image/val/"
labels_image = "data/x-ai_data.csv"

## Create Subfolder

Use the fonction "subfolders" to do the same thing as the cells below, DO NOT run it twice to avoid duplicates

In [ ]:
train = labels_image.loc[labels_image['split']=='train']
val = labels_image.loc[labels_image['split']=='validation']

In [ ]:
train0 = train.loc[train['class']==0]
train1 = train.loc[train['class']==1]

val0 = val.loc[val['class']==0]
val1 = val.loc[val['class']==1]


In [ ]:
#Move TRAIN images labeled 0 to the correct folder

for i in train0.index:
    im = cv2.imread(img_folder+train0.loc[i,'filename'])
    cv2.imwrite(train_img + '0/' +train0.loc[i,'filename'], im)
#Move TRAIN images labeled 1 to the correct folder
for i in train1.index:
    im = cv2.imread(img_folder+train1.loc[i,'filename'])
    cv2.imwrite(train_img+'1/' +train1.loc[i,'filename'], im)

In [ ]:
#Move val images labeled 0 to the correct folder
for i in val0.index:
    im = cv2.imread(img_folder+val0.loc[i,'filename'])
    cv2.imwrite(val_img + '0/' +val0.loc[i,'filename'], im)
#Move val images labeled 1 to the correct folder
for i in val1.index:
    im = cv2.imread(img_folder+val1.loc[i,'filename'])
    cv2.imwrite(val_img+'1/' +val1.loc[i,'filename'], im)

In [ ]:
import shutil

#Remove hidden file
shutil.rmtree("image/train/.ipynb_checkpoints")
shutil.rmtree("image/val/.ipynb_checkpoints")

## Model

In [ ]:
## Train and Val dataset
train_ds = train_set(train_img, config.image_size, config.batch_size)
val_ds = val_set(val_img, config.image_size, config.batch_size)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
## Model
model = make_model(input_shape=config.image_size + (3,), num_classes=2)

In [ ]:
train_model(model, train_ds, val_ds, config.number_epochs)

In [ ]:
#visualize data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.1),
        layers.RandomContrast([0,1]),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
    ]
)
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")